In [1]:
from openfisca_us import Microsimulation, reforms
from openfisca_us.api import *
import numpy as np
import pandas as pd

In [2]:
# Calculate poverty rate
baseline = Microsimulation(year=2020)
baseline.calc("in_poverty", map_to="person").mean()

0.11652875394831709

In [3]:
from openfisca_us.entities import *


class basic_income(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "Basic income"

    def formula(tax_unit, period, parameters):
        # Extract FPG parameter
        fpg_params = parameters(period).poverty.fpg
        # Includes first_person and additional_person
        # Extract tax unit number of people
        nb_people = tax_unit.nb_persons()
        # Calculate FPG
        fpg = (
            fpg_params.first_person.contiguous_US
            + (nb_people - 1) * fpg_params.additional_person.contiguous_US
        )
        # Extract taxable income
        taxable_income = tax_unit("taxable_income", period)
        # Calculate basic income phased out at 50%
        return np.maximum(fpg - taxable_income * 0.5, 0)
        # TODO: Make 50% a parameter

In [4]:
class taxable_income(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR

In [5]:
class SPM_unit_net_income(Variable):
    value_type = float
    entity = SPMUnit
    label = u"SPM unit net income"
    definition_period = YEAR

    def formula(spm_unit, period):
        INCOME_COMPONENTS = [
            "SPM_unit_total_income",
            "SPM_unit_SNAP",
            "SPM_unit_capped_housing_subsidy",
            "SPM_unit_school_lunch_subsidy",
            "SPM_unit_energy_subsidy",
            "SPM_unit_WIC",
        ]
        EXPENSE_COMPONENTS = [
            "SPM_unit_FICA",
            "SPM_unit_federal_tax",
            "SPM_unit_state_tax",
            "SPM_unit_capped_work_childcare_expenses",
            "SPM_unit_medical_expenses",
        ]
        income = add(spm_unit, period, *INCOME_COMPONENTS)
        expense = add(spm_unit, period, *EXPENSE_COMPONENTS)
        basic_income = sum_contained_tax_units(
            "basic_income", spm_unit, period
        )
        return income - expense + basic_income

In [6]:
reform = (
    reforms.restructure(SPM_unit_net_income),
    reforms.new_variable(basic_income),
    reforms.new_variable(taxable_income),
)
reform_sim = Microsimulation(reform, year=2020)
from openfisca_us_data import RawCPS

taxable_income_values = RawCPS.load(2020, "tax_unit").TAX_INC.values
reform_sim.simulation.set_input("taxable_income", 2020, taxable_income_values)

In [7]:
df = reform_sim.df(
    ["SPM_unit_net_income", "taxable_income", "in_poverty", "basic_income"]
)

In [8]:
df.basic_income.groupby(df.taxable_income.decile_rank()).mean()

1.0     17615.118505
2.0     18213.645279
3.0     17982.893939
4.0     12921.723542
5.0      7550.313452
6.0      4565.691330
7.0      3264.406229
8.0      2298.077669
9.0      1967.494285
10.0     1499.863151
dtype: float64